In [ ]:
import MDAnalysis as mda
import matplotlib.pyplot as plt
import numpy as np
from MDAnalysis.analysis import rms
import MDAnalysis.analysis.hbonds
import pandas as pd
import seaborn as sns
from numpy.linalg import norm

u1 = mda.Universe('pull-v-0-1.gro','pull-v-0-1.xtc')
u01 = mda.Universe('pull-v-0-01.gro','pull-v-0-01.xtc')
u001 = mda.Universe('pull-v-0-001_centre.gro','pull-v-0-001_centre.xtc')
u0001 = mda.Universe('pull-v-0-0001_centre.gro','pull-v-0-0001_centre.xtc')

different_traj = [u1,u01,u001,u0001]
different_velocities = ['u1','u01','u001','u0001']

Multiple_traj = {k: j for k,j in zip(different_velocities,different_traj)}

'''
#######################################################################################
#######################################################################################
#######################################################################################
#Hydrogen Bonds

h = MDAnalysis.analysis.hbonds.HydrogenBondAnalysis(u0001,'protein')
h.run()
count = h.count_by_time()
df = pd.DataFrame.from_records(count)
df.plot(x='time', y='count')
plt.show()
#df.plot()
#plt.show()
#######################################################################################
#######################################################################################
#######################################################################################
'''

#######################################################################################
#######################################################################################
#######################################################################################

#Radius of Gyration- 
Multiple_Rgyr = {k:[] for k in different_velocities}

for vel in different_velocities:
    frames_num = len(Multiple_traj[vel].trajectory)
    protein = Multiple_traj[vel].select_atoms("protein")
    for ts in Multiple_traj[vel].trajectory:
        Multiple_Rgyr[vel].append((Multiple_traj[vel].trajectory.time / frames_num, protein.radius_of_gyration()))
    Multiple_Rgyr[vel] = np.array(Multiple_Rgyr[vel])




ax = plt.subplot(111)
for i in different_velocities:
    a= (len(Multiple_Rgyr[i][:,0]))
    ax.plot(np.linspace(0,500,num = a), Multiple_Rgyr[i][:,1], '--', lw=2, label=r"Velocity: 0.{}nm/ps".format(i[1:])) #sim time is number of frames. Then normalise
                                                                                                              #with respect to #frames.

#ax.legend()
ax.set_xlabel(r"Time ($ps$)")
ax.set_ylabel(r"Radius of gyration $R_G$ ($\AA$)")
plt.legend(loc = 'upper left',prop={'size': 8.5})

plt.draw()
plt.xlim([0,500])
plt.ylim([6.6,16])
ax.figure.savefig("Rgyr.pdf")
plt.show()

Average_Rgyr = {k: [] for k in different_velocities}
Error_Rgyr = {k: [] for k in different_velocities}

for vel in different_velocities:
    Average_Rgyr[vel] = np.mean(Multiple_Rgyr[vel][:,1]**2) * 6
    Error_Rgyr[vel] = np.std(Multiple_Rgyr[vel][:,1]**2) * 6
    print ('Velocity: {}, Average: {}, Error: {}'.format(vel[1:],Average_Rgyr[vel],Error_Rgyr[vel]))




#######################################################################################
#######################################################################################
#######################################################################################

Multiple_end_to_end = {k:[] for k in different_velocities}
for vel in different_velocities:
    frames_num = len(Multiple_traj[vel].trajectory)

    for i in range(frames_num):
        Multiple_traj[vel].trajectory[i] #tells universe what frame it should be on
        current_CA = Multiple_traj[vel].select_atoms('name CA') #selects CA atoms
        current_positions = current_CA.positions.copy() #copies the positions of CA atoms
        start_CA = current_positions[0] #first CA atom position
        end_CA = current_positions[-1] #last CA atom position
        Multiple_end_to_end[vel].append(norm(start_CA - end_CA)) #Magnitude of vector between them

'''
plt.clf()
ax = plt.subplot(111)
for i in different_velocities:
    a = len(Multiple_end_to_end[i])
    ax.plot(np.linspace(0,500,num = a)/a, Multiple_end_to_end[i], '--', lw=2, label="End-to-end distance: {}".format(i[1:]))
ax.set_xlabel("Normalised time")
ax.set_ylabel(r"End-to-end distance")
ax.figure.savefig("End-to-end_distance.pdf")
plt.clf()
'''

#######################################################################################
#######################################################################################
#######################################################################################

#average
Multiple_average = {k:None for k in different_velocities}
Multiple_error = {k:None for k in different_velocities}

for i in different_velocities:

    Multiple_average[i] = np.mean(np.array(Multiple_end_to_end[i])**2)
    Multiple_error[i] = np.std(np.array(Multiple_end_to_end[i])**2)
    print ('Velocity: {}, Average: {}, Error: {}'.format(i[1:],Multiple_average[i],Multiple_error[i]))



#######################################################################################
#######################################################################################
#######################################################################################
#Histograms
plt.clf()
fig, ax = plt.subplots(nrows=2, ncols=2)

i = 0
for row in ax:
    for col in row:

        maxi= np.max(Multiple_end_to_end[different_velocities[i]])
        mini=np.min(Multiple_end_to_end[different_velocities[i]])
        numbeer = len(Multiple_end_to_end[different_velocities[i]])
        x,bins,_ = col.hist(Multiple_end_to_end[different_velocities[i]],label = 'Velocity: \n0.{}nm/ps'.format(different_velocities[i][1:]),density = True,bins = np.linspace(mini,maxi,num = int(numbeer/2) ))
        print(bins[np.argmax(x)])
        col.set_xlabel('End-to-end distance (nm)')
        if i < 2:
            col.xaxis.set_label_position('top')
            col.xaxis.tick_top()
        col.set_ylabel('Frequency')
        if i == 1 or i == 3:
            col.set_ylabel( 'Frequency',rotation=270)
            col.yaxis.labelpad = 15
            col.yaxis.set_label_position('right')
            col.yaxis.tick_right()

        if i == 2:
            y = sns.distplot(Multiple_end_to_end[different_velocities[i]],ax = col,hist = False,label = 'Univariate\ndist.')
        else:
            y = sns.distplot(Multiple_end_to_end[different_velocities[i]],ax = col,hist = False,label = 'Univariate dist.')
        col.legend(prop={'size': 6.5},loc = 'best')
        handles,labels = col.get_legend_handles_labels()
        handles = [handles[0], handles[1]]
        labels = [labels[0], labels[1]]
        i += 1
fig.savefig("Histograms.pdf")
plt.show()
